# Training an NLP-based Model

In this notebook, we will explore the training of an NLP-based model using pre-defined architectures from `huggingface/transformers`.

Archai implements abstract base classes that defines the expected behavior of some classes, such as datasets (`DatasetProvider`) and trainers (`TrainerBase`). Additionally, we offer boilerplate classes for the most common frameworks, such as a `DatasetProvider` compatible with `huggingface/datasets` and a `TrainerBase` compatible with `huggingface/transformers`.

## Loading and Encoding the Data

In [ ]:
from transformers import AutoTokenizer, DataCollatorForLanguageModeling
from archai.datasets.nlp.hf_dataset_provider import HfHubDatasetProvider
from archai.datasets.nlp.hf_dataset_provider_utils import tokenize_dataset

tokenizer = AutoTokenizer.from_pretrained("Salesforce/codegen-350M-mono", model_max_length=1024)
tokenizer.pad_token = tokenizer.eos_token

collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
dataset_provider = HfHubDatasetProvider(dataset="wikitext", subset="wikitext-103-raw-v1")

train_dataset = dataset_provider.get_train_dataset()
eval_dataset = dataset_provider.get_val_dataset()

encoded_train_dataset = train_dataset.map(tokenize_dataset, batched=True, fn_kwargs={"tokenizer": tokenizer})
encoded_eval_dataset = eval_dataset.map(tokenize_dataset, batched=True, fn_kwargs={"tokenizer": tokenizer})

## Defining the Model

In [ ]:
from transformers import CodeGenConfig, CodeGenForCausalLM

config = CodeGenConfig(
    n_positions=1024,
    n_embd=768,
    n_layer=12,
    n_head=12,
    rotary_dim=16,
    bos_token_id=0,
    eos_token_id=0,
    vocab_size=50295,
)
model = CodeGenForCausalLM(config=config)

## Running the Trainer

In [ ]:
from transformers import TrainingArguments
from archai.trainers.nlp.hf_trainer import HfTrainer

training_args = TrainingArguments(
    "hf-codegen",
    evaluation_strategy="steps",
    logging_steps=10,
    eval_steps=25,
    per_device_train_batch_size=1,
    learning_rate=0.01,
    weight_decay=0.1,
    max_steps=25,
)
trainer = HfTrainer(
    model=model,
    args=training_args,
    data_collator=collator,
    train_dataset=encoded_train_dataset,
    eval_dataset=encoded_eval_dataset,
)

trainer.train()